<a href="https://colab.research.google.com/github/sravanthi200719/HDS5210_InClass/blob/master/Copy_of_midterm_ipynb_week_8_assignment_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mid-term for HDS5210
Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns: NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns: NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns: HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [1]:
import doctest
import json
import csv

# Step 1: Calculate BMI

In [2]:
def calculate_bmi(weight_kg, height_m):
    """
    Calculate the Body Mass Index (BMI) given the weight and height.

    Parameters:
    weight_kg (float): The weight in kilograms.
    height_m (float): The height in meters.

    Returns:
    float: The calculated BMI rounded to one decimal place.
    """
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 1)

In [3]:
def interpret_bmi(bmi):
    """
    Interpret the BMI value and categorize it based on standard weight status categories.

    Parameters:
    bmi (float): The BMI value to interpret.

    Returns:
    str: The category of the BMI, which can be one of the following:
         - "Underweight" if BMI is less than 18.5.
         - "Normal weight" if BMI is between 18.5 and 24.9.
         - "Overweight" if BMI is between 25 and 29.9.
         - "Obese" if BMI is 30 or above.
    """
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal weight"
    elif 25 <= bmi < 30:
        return "Overweight"
    else:
        return "Obese"

def display_bmi_result(weight, height, bmi, category):
    """
    Display the BMI calculation results in a formatted and readable manner.

    Parameters:
    weight (float): The weight of the person in kilograms.
    height (float): The height of the person in meters.
    bmi (float): The calculated BMI value.
    category (str): The BMI category based on the BMI interpretation.

    Returns:
    None: This function only prints the formatted BMI result.
    """
    print("\nBMI Calculation Results")
    print("-----------------------")
    print(f"Weight: {weight:.1f} kg")
    print(f"Height: {height:.2f} m")
    print(f"BMI: {bmi:.1f}")
    print(f"Category: {category}")
    print("-----------------------")


In [4]:
### # Test Case 1: Normal weight
print("Test Case 1: Normal weight")
weight1, height1 = 70, 1.75
bmi1 = calculate_bmi(weight1, height1)
category1 = interpret_bmi(bmi1)
display_bmi_result(weight1, height1, bmi1, category1)

Test Case 1: Normal weight

BMI Calculation Results
-----------------------
Weight: 70.0 kg
Height: 1.75 m
BMI: 22.9
Category: Normal weight
-----------------------


In [5]:
### # Test Case 2: Overweight
print("\nTest Case 2: Overweight")
weight2, height2 = 90, 1.80
bmi2 = calculate_bmi(weight2, height2)
category2 = interpret_bmi(bmi2)
display_bmi_result(weight2, height2, bmi2, category2)


Test Case 2: Overweight

BMI Calculation Results
-----------------------
Weight: 90.0 kg
Height: 1.80 m
BMI: 27.8
Category: Overweight
-----------------------


In [6]:
#### # Test Case 3: Underweight
print("\nTest Case 3: Underweight")
weight3, height3 = 50, 1.70
bmi3 = calculate_bmi(weight3, height3)
category3 = interpret_bmi(bmi3)
display_bmi_result(weight3, height3, bmi3, category3)



Test Case 3: Underweight

BMI Calculation Results
-----------------------
Weight: 50.0 kg
Height: 1.70 m
BMI: 17.3
Category: Underweight
-----------------------


### Step 2: Calculate BODE Score

In [7]:
# Constants for BODE calculation
DYSPNEA_SCORES = {
    'No dyspnea': 0,
    'Dyspnea when walking fast': 1,
    'Dyspnea when walking on level ground': 2,
    'Dyspnea after 100m': 3,
    'Too dyspneic to leave the house': 4
}

def calculate_bmi(weight_kg, height_m):
    """
    Calculate Body Mass Index (BMI).

    Parameters:
    weight_kg (float): The weight in kilograms.
    height_m (float): The height in meters.

    Returns:
    float: The BMI value, rounded to two decimal places.
    """
    return round(weight_kg / (height_m ** 2), 2)


In [8]:
# Constants for BODE calculation
DYSPNEA_SCORES = {
    'No dyspnea': 0,
    'Dyspnea when walking fast': 1,
    'Dyspnea when walking on level ground': 2,
    'Dyspnea after 100m': 3,
    'Too dyspneic to leave the house': 4  # Scores representing dyspnea severity
}

def calculate_bmi(weight_kg, height_m):
    """
    Calculate Body Mass Index (BMI).

    Parameters:
        weight_kg (float): Weight in kilograms.
        height_m (float): Height in meters.

    Returns:
        float: The BMI, rounded to two decimal places.
    """
    return round(weight_kg / (height_m ** 2), 2)

def calculate_bode_score(bmi, fev_pct, dyspnea_desc, distance_meters):
    """
    Calculate the BODE score, predicting the risk of death from COPD.

    Parameters:
        bmi (float): BMI value.
        fev_pct (int): FEV1 percentage.
        dyspnea_desc (str): Description of dyspnea severity.
        distance_meters (int): Distance walked in meters during a 6-minute test.

    Returns:
        int: The calculated BODE score.
    """
    score = 0

    # FEV1% contribution
    if fev_pct >= 65:
        score += 0
    elif fev_pct >= 50:
        score += 1
    elif fev_pct >= 36:
        score += 2
    else:
        score += 3

    # BMI contribution
    score += 0 if bmi > 21 else 1

    # Dyspnea score
    score += DYSPNEA_SCORES.get(dyspnea_desc, 0)

    # Distance walked contribution
    if distance_meters > 350:
        score += 0
    elif distance_meters >= 250:
        score += 1
    elif distance_meters >= 150:
        score += 2
    else:
        score += 3

    return score


In [9]:
### # test case 1.
bmi = calculate_bmi(80, 1.80)  # Weight: 80 kg, Height: 1.80 meters
bode_score = calculate_bode_score(bmi, 55, 'Dyspnea when walking fast', 300)  # FEV1%: 55, Dyspnea: 'Dyspnea when walking fast', Distance: 300 meters

# Print the BODE score
print(f"BODE Score: {bode_score}")


BODE Score: 3


In [10]:
### test case 1
bmi = calculate_bmi(80, 1.80) # Weight: 80 kg, Height: 1.80 meters bode_score = calculate_bode_score(bmi, 55, 'Dyspnea when walking fast', 300) # FEV1%: 55, Dyspnea: 'Dyspnea when walking fast', Distance: 300 meters

print(f"BODE Score: {bode_score}")

BODE Score: 3


In [11]:
### test case 2
bmi = calculate_bmi(60, 1.70)  # Weight: 60 kg, Height: 1.70 meters
bode_score = calculate_bode_score(bmi, 40, 'Dyspnea after 100m', 150)  # FEV1%: 40, Dyspnea: 'Dyspnea after 100m', Distance: 150 meters

print(f"BODE Score: {bode_score}")


BODE Score: 8


In [12]:
### # Test case 3
bmi = calculate_bmi(70, 1.75)  # Weight: 70 kg, Height: 1.75 meters
bode_score = calculate_bode_score(bmi, 55, 'Dyspnea after 1 km', 300)  # FEV1%: 55, Dyspnea: 'Dyspnea after 1 km', Distance: 300 meters

print(f"BODE Score: {bode_score}")

BODE Score: 2


### Step 3: Calculate BODE Risk

In [13]:
def calculate_bode_risk(bode_score):
    """
    Classify the BODE score into a corresponding risk category for patients with chronic obstructive pulmonary disease (COPD).

    Parameters:
    bode_score (int): The calculated BODE score (a sum of component scores that assess disease severity).

    Returns:
    str: The risk category based on the BODE score, which can be one of the following:
         - "Low" for scores of 0 to 2.
         - "Moderate" for scores of 3 to 4.
         - "High" for scores of 5 to 6.
         - "Very High" for scores of 7 or above.
    """
    if bode_score <= 2:
        return "Low"
    elif bode_score <= 4:
        return "Moderate"
    elif bode_score <= 6:
        return "High"
    else:
        return "Very High"


In [14]:
### test case 1
bmi = calculate_bmi(70, 1.75)
bode_score = calculate_bode_score(bmi, 60, 'Dyspnea when walking fast', 300)
bode_risk = calculate_bode_risk(bode_score)

print(f"BODE Score: {bode_score}, BODE Risk: {bode_risk}")

BODE Score: 3, BODE Risk: Moderate


In [15]:
### test case 2
bmi = calculate_bmi(60, 1.65)  # Weight: 60 kg, Height: 1.65 meters
bode_score = calculate_bode_score(bmi, 40, 'Dyspnea after 100m', 150)  # FEV1%: 40, Dyspnea: 'Dyspnea after 100m', Distance: 150 meters
bode_risk = calculate_bode_risk(bode_score)

print(f"BODE Score: {bode_score}, BODE Risk: {bode_risk}")


BODE Score: 7, BODE Risk: Very High


In [16]:
### test case 3
bmi = calculate_bmi(70, 1.75)  # Weight: 70 kg, Height: 1.75 meters
bode_score = calculate_bode_score(bmi, 60, 'Dyspnea when walking fast', 300)  # FEV1%: 60, Dyspnea: 'Dyspnea when walking fast', Distance: 300 meters
bode_risk = calculate_bode_risk(bode_score)

print(f"BODE Score: {bode_score}, BODE Risk: {bode_risk}")



BODE Score: 3, BODE Risk: Moderate


### Step 4: Load Hospital Data

In [17]:
import json

def load_hospital_data(file_path):
    """
    Load hospital data from a JSON file.

    Parameters:
    file_path (str): The path to the JSON file containing hospital data.

    Returns:
    dict or list: The data loaded from the JSON file, which can be a list of dictionaries or a single dictionary,
    depending on the structure of the JSON file.

    Raises:
    FileNotFoundError: If the file specified by file_path is not found.
    JSONDecodeError: If the file is not a valid JSON format.
    """
    with open(file_path, 'r') as file:
        hospital_data = json.load(file)


In [18]:
import json

def load_hospital_data(file_path):                #load hospital data
    with open(file_path, 'r') as file:
        hospital_data = json.load(file)
    return hospital_data

hospital_data = load_hospital_data('hospitals (1).json')
print(json.dumps(hospital_data, indent=2))


[
  {
    "system": "BJC",
    "hospitals": [
      {
        "name": "BJC",
        "beds": 2000
      },
      {
        "name": "BJC WEST COUNTY",
        "beds": 1000
      },
      {
        "name": "MISSOURI BAPTIST",
        "beds": 800
      }
    ]
  },
  {
    "system": "SSM",
    "hospitals": [
      {
        "name": "SAINT LOUIS UNIVERSITY",
        "beds": 1000
      },
      {
        "name": "ST.MARY'S",
        "beds": 500
      }
    ]
  },
  {
    "system": "ST.LUKE'S",
    "hospitals": [
      {
        "name": "ST.LUKE'S",
        "beds": 800
      }
    ]
  }
]


In [19]:
### TEST CASE 1.
# Test loading hospital data
hospital_data = load_hospital_data('hospitals (1).json')

# Expected output: hospital_data should be a list with dictionaries for each hospital system
print(hospital_data)


[{'system': 'BJC', 'hospitals': [{'name': 'BJC', 'beds': 2000}, {'name': 'BJC WEST COUNTY', 'beds': 1000}, {'name': 'MISSOURI BAPTIST', 'beds': 800}]}, {'system': 'SSM', 'hospitals': [{'name': 'SAINT LOUIS UNIVERSITY', 'beds': 1000}, {'name': "ST.MARY'S", 'beds': 500}]}, {'system': "ST.LUKE'S", 'hospitals': [{'name': "ST.LUKE'S", 'beds': 800}]}]


### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [20]:
import json

# Constants for dyspnea scoring based on difficulty in breathing
DYSPNEA_SCORES = {
    'No dyspnea': 0,
    'Dyspnea when walking fast': 1,
    'Dyspnea when walking on level ground': 2,
    'Dyspnea after 100m': 3,
    'Too dyspneic to leave the house': 4
}
"""
DYSPNEA_SCORES:
Maps levels of dyspnea to numerical scores for BODE calculation. Keys indicate severity:
- 'No dyspnea': 0
- 'Dyspnea when walking fast': 1
- 'Dyspnea when walking on level ground': 2
- 'Dyspnea after 100m': 3
- 'Too dyspneic to leave the house': 4
"""

def calculate_bmi(weight_kg, height_m):
    """
    Calculate Body Mass Index (BMI).

    Parameters:
        weight_kg (float): Weight in kilograms.
        height_m (float): Height in meters.

    Returns:
        float: BMI rounded to two decimal places.
    """
    return round(weight_kg / (height_m ** 2), 2)

def calculate_bode_score(bmi, fev_pct, dyspnea_desc, distance_meters):
    """
    Calculate the BODE score, predicting risk in COPD patients.

    Factors considered:
    - BMI
    - FEV1% (percentage of predicted value)
    - Dyspnea severity
    - Distance walked in 6 minutes

    Parameters:
        bmi (float): BMI value.
        fev_pct (int): FEV1 percentage.
        dyspnea_desc (str): Description of dyspnea severity.
        distance_meters (int): Distance walked during a test.

    Returns:
        int: Calculated BODE score.
    """
    score = 0

    # FEV1% contribution
    if fev_pct >= 65:
        score += 0
    elif fev_pct >= 50:
        score += 1
    elif fev_pct >= 36:
        score += 2
    else:
        score += 3

    # BMI contribution
    score += 0 if bmi > 21 else 1

    # Dyspnea score
    score += DYSPNEA_SCORES.get(dyspnea_desc, 0)

    # Distance walked contribution
    if distance_meters > 350:
        score += 0
    elif distance_meters >= 250:
        score += 1
    elif distance_meters >= 150:
        score += 2
    else:
        score += 3

    return score

def calculate_bode_risk(bode_score):
    """
    Classify BODE score into risk categories.

    Parameters:
        bode_score (int): The calculated BODE score.

    Returns:
        str: Risk category based on the BODE score:
             - "Low" (0-2)
             - "Moderate" (3-4)
             - "High" (5-6)
             - "Very High" (7+)
    """
    if bode_score <= 2:
        return "Low"
    elif bode_score <= 4:
        return "Moderate"
    elif bode_score <= 6:
        return "High"
    else:
        return "Very High"


In [21]:
import json

def load_hospital_data(file_path):
    """
    Load hospital data from a JSON file.

    Parameters:
    file_path (str): The path to the JSON file containing hospital data.

    Returns:
    list: A list of dictionaries representing hospitals and their details.

    Raises:
    FileNotFoundError: If the file specified by file_path is not found.
    JSONDecodeError: If the file is not a valid JSON format.
    """
    with open(file_path, 'r') as file:
        hospital_data = json.load(file)
    return hospital_data

def process_patients(patients, hospitals):
    """
    Process patient data and calculate BODE score, risk categories, and hospital statistics.

    This function processes each patient's data, calculating the BODE score and risk based on their BMI,
    FEV1 percentage, dyspnea severity, and distance walked. It then compiles statistics for each hospital.

    Parameters:
    patients (list): A list of dictionaries representing patient data.
                     Each patient dictionary contains keys like 'WEIGHT_KG', 'HEIGHT_M', 'fev_pct',
                     'dyspnea_description', 'distance_in_meters', and 'hospital'.
    hospitals (list): A list of dictionaries representing hospital systems, where each hospital system contains
                      a list of hospitals with details like the number of beds.

    Returns:
    dict: A dictionary of hospital statistics with the following structure:
          - 'total_score': The cumulative BODE score for all patients treated at the hospital.
          - 'total_cases': The total number of patients treated at the hospital.
          - 'risk_count': A dictionary counting patients in each BODE risk category ('Low', 'Moderate', 'High', 'Very High').
          - 'beds': The number of available beds at the hospital.

    Example:
    The returned dictionary for each hospital might look like:
    {
        "Hospital A": {
            "total_score": 25,
            "total_cases": 5,
            "risk_count": {"Low": 2, "Moderate": 1, "High": 1, "Very High": 1},
            "beds": 150
        },
        "Hospital B": {
            "total_score": 12,
            "total_cases": 3,
            "risk_count": {"Low": 1, "Moderate": 1, "High": 0, "Very High": 1},
            "beds": 200
        }
    }

    Raises:
    KeyError: If the expected keys are missing from the patient or hospital data.
    """
    hospital_stats = {}

    for patient in patients:
        # Calculate BMI for the patient
        bmi = calculate_bmi(float(patient['WEIGHT_KG']), float(patient['HEIGHT_M']))

        # Calculate BODE score based on BMI, FEV1%, dyspnea, and distance walked
        bode_score = calculate_bode_score(
            bmi,
            float(patient['fev_pct']),
            patient['dyspnea_description'],
            float(patient['distance_in_meters'])
        )

        # Determine the BODE risk category based on the BODE score
        bode_risk = calculate_bode_risk(bode_score)

        # Store the BODE score and risk category in the patient's data
        patient['BODE_SCORE'] = bode_score
        patient['BODE_RISK'] = bode_risk

        # Get the hospital name from the patient data
        hospital_name = patient['hospital']

        # Initialize the hospital's stats if not already present in the dictionary
        if hospital_name not in hospital_stats:
            hospital_stats[hospital_name] = {
                'total_score': 0,
                'total_cases': 0,
                'risk_count': {'Low': 0, 'Moderate': 0, 'High': 0, 'Very High': 0},
                'beds': next((hospital['beds'] for sys in hospitals
                              for hospital in sys['hospitals']
                              if hospital['name'] == hospital_name), 0)
            }

        # Update hospital statistics with the patient's BODE score and risk category
        hospital_stats[hospital_name]['total_score'] += bode_score
        hospital_stats[hospital_name]['total_cases'] += 1
        hospital_stats[hospital_name]['risk_count'][bode_risk] += 1

    return hospital_stats


In [22]:
def calculate_average_stats(hospital_stats):
    for hospital_name, stats in hospital_stats.items():
        stats['avg_score'] = stats['total_score'] / stats['total_cases'] if stats['total_cases'] > 0 else 0
        stats['avg_risk'] = max(stats['risk_count'], key=stats['risk_count'].get) if stats['total_cases'] > 0 else 'N/A'
    return hospital_stats

def main():
    patients = [
        {
            'NAME': 'John Doe', 'WEIGHT_KG': 70, 'HEIGHT_M': 1.75, 'fev_pct': 60,
            'dyspnea_description': 'Dyspnea when walking fast', 'distance_in_meters': 300,
            'hospital': 'BJC'
        },
        {
            'NAME': 'Jane Smith', 'WEIGHT_KG': 60, 'HEIGHT_M': 1.65, 'fev_pct': 40,
            'dyspnea_description': 'Dyspnea after 100m', 'distance_in_meters': 150,
            'hospital': 'MISSOURI BAPTIST'
        }
    ]

    hospitals = load_hospital_data('hospitals (1).json')
    hospital_stats = process_patients(patients, hospitals)
    hospital_stats = calculate_average_stats(hospital_stats)

    for hospital, stats in hospital_stats.items():
        print(f'Hospital: {hospital}')
        print(f'  Total COPD Cases: {stats["total_cases"]}')
        print(f'  Average BODE Score: {stats["avg_score"]}')
        print(f'  Most Common BODE Risk: {stats["avg_risk"]}')
        print(f'  Beds: {stats["beds"]}')
        print('')

if __name__ == '__main__':
    main()


Hospital: BJC
  Total COPD Cases: 1
  Average BODE Score: 3.0
  Most Common BODE Risk: Moderate
  Beds: 2000

Hospital: MISSOURI BAPTIST
  Total COPD Cases: 1
  Average BODE Score: 7.0
  Most Common BODE Risk: Very High
  Beds: 800



In [23]:
### test case 1
# Test patient data
patients = [
    {
        'NAME': 'John Doe', 'WEIGHT_KG': 70, 'HEIGHT_M': 1.75, 'fev_pct': 60, 'dyspnea_description': 'Dyspnea when walking fast', 'distance_in_meters': 300, 'hospital': 'BJC'
    },
    {
        'NAME': 'Jane Smith', 'WEIGHT_KG': 60, 'HEIGHT_M': 1.65, 'fev_pct': 40, 'dyspnea_description': 'Dyspnea after 100m', 'distance_in_meters': 150, 'hospital': 'MISSOURI BAPTIST'
    }
]

# Test hospital data
hospitals = [
    {
        "system": "BJC",
        "hospitals": [
            {"name": "BJC", "beds": 2000},
            {"name": "BJC WEST COUNTY", "beds": 1000},
            {"name": "MISSOURI BAPTIST", "beds": 800}
        ]
    }
]

# Process the test patients and hospital data
hospital_stats = process_patients(patients, hospitals)
hospital_stats = calculate_average_stats(hospital_stats)

##  output for hospital BJC
print(f'Hospital: BJC')
print(f'  Total COPD Cases: {hospital_stats["BJC"]["total_cases"]}')
print(f'  Average BODE Score: {hospital_stats["BJC"]["avg_score"]}')
print(f'  Most Common BODE Risk: {hospital_stats["BJC"]["avg_risk"]}')
print(f'  Beds: {hospital_stats["BJC"]["beds"]}')

## output for hospital MISSOURI BAPTIST
print(f'Hospital: MISSOURI BAPTIST')
print(f'  Total COPD Cases: {hospital_stats["MISSOURI BAPTIST"]["total_cases"]}')
print(f'  Average BODE Score: {hospital_stats["MISSOURI BAPTIST"]["avg_score"]}')
print(f'  Most Common BODE Risk: {hospital_stats["MISSOURI BAPTIST"]["avg_risk"]}')
print(f'  Beds: {hospital_stats["MISSOURI BAPTIST"]["beds"]}')


Hospital: BJC
  Total COPD Cases: 1
  Average BODE Score: 3.0
  Most Common BODE Risk: Moderate
  Beds: 2000
Hospital: MISSOURI BAPTIST
  Total COPD Cases: 1
  Average BODE Score: 7.0
  Most Common BODE Risk: Very High
  Beds: 800


In [24]:
### TEST CASE 2  #
#Test patient data
patients = [
    {
        'NAME': 'Alice Green', 'WEIGHT_KG': 85, 'HEIGHT_M': 1.70, 'fev_pct': 50, 'dyspnea_description': 'Dyspnea after 100m', 'distance_in_meters': 100, 'hospital': 'ST.LUKE\'S'
    },
    {
        'NAME': 'Bob White', 'WEIGHT_KG': 90, 'HEIGHT_M': 1.80, 'fev_pct': 35, 'dyspnea_description': 'Too dyspneic to leave the house', 'distance_in_meters': 50, 'hospital': 'ST.LUKE\'S'
    }
]

# Test hospital data
hospitals = [
    {
        "system": "ST.LUKE'S",
        "hospitals": [
            {"name": "ST.LUKE'S", "beds": 800}
        ]
    }
]

# Process the test patients and hospital data
hospital_stats = process_patients(patients, hospitals)
hospital_stats = calculate_average_stats(hospital_stats)

# Expected output for hospital ST.LUKE'S
print("Hospital: ST.LUKE'S")
print("  Total COPD Cases: {}".format(hospital_stats["ST.LUKE'S"]["total_cases"]))
print("  Average BODE Score: {}".format(hospital_stats["ST.LUKE'S"]["avg_score"]))
print("  Most Common BODE Risk: {}".format(hospital_stats["ST.LUKE'S"]["avg_risk"]))
print("  Beds: {}".format(hospital_stats["ST.LUKE'S"]["beds"]))



Hospital: ST.LUKE'S
  Total COPD Cases: 2
  Average BODE Score: 8.5
  Most Common BODE Risk: Very High
  Beds: 800


In [25]:
patient_csv = "patient.csv"
hospital_json = "hospitals.json"

patient_output_file = "patient_output.csv"
hospital_output_file = "hospital_output.csv"

###
# Your logic here
###
patient_results = []
hospital_output_list = []

#Write Patient_output.csv
with open(patient_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(patient_results)
#Write Hospital_output.csv
with open(hospital_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hospital_output_list)